In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
print(os.listdir("../input/histopathologic-cancer-detection"))

#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../input/histopathologic-cancer-detection'

In [23]:
import PIL
import cv2
import PIL.Image 
import shutil
import itertools
import tensorflow as tf 
import matplotlib.pyplot as  plt
import  tensorflow_datasets as tfds

!pip install tensorflow
!pip install keras
!pip install Pillow

import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from keras.applications.vgg16 import VGG16
from keras.optimizers import Adam

from keras.models import Sequential
from keras.applications.resnet50 import ResNet50
from keras import layers
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, BatchNormalization, Flatten, Activation
from keras import backend as k 
from tensorflow.keras import layers
import tensorflow_hub as hub

ModuleNotFoundError: No module named 'cv2'

In [ ]:
labels = pd.read_csv('../input/histopathologic-cancer-detection/train_labels.csv')
print(labels.head())

In [ ]:
labels['label'].value_counts()

In [ ]:
labels['label']=labels['label'].apply(lambda x: str(x))

In [ ]:
print(len(labels))

In [1]:
train= "../input/histopathologic-cancer-detection/train"
#test= '../input/histopathologic-cancer-detection/test'
print(len(os.listdir(train)))

NameError: name 'os' is not defined

In [2]:
train_img= plt.imread('../input/histopathologic-cancer-detection/train/' + labels.iloc[0]['id'] + '.tif')

print('Image Shape:', train_img.shape)

NameError: name 'plt' is not defined

In [3]:
for i in range(5) :
    train_img= plt.imread('../input/histopathologic-cancer-detection/train/' + labels.iloc[i]['id'] + '.tif')
    print(labels.iloc[i]['label'])    
    plt.imshow(train_img)
    plt.show()

NameError: name 'plt' is not defined

**DATA GENERATORS**


In [3]:
y= labels['label']
from sklearn.model_selection import train_test_split

trainx, valx = train_test_split(labels,test_size=0.2, random_state= 99, stratify= y)
print(trainx.shape)
print(valx.shape)
print(trainx['label'].value_counts())
print(valx['label'].value_counts())

print(len(trainx['id']))
print(len(valx['id']))

NameError: name 'labels' is not defined

In [4]:
train='../input/histopathologic-cancer-detection/train'
valid= '../input/histopathologic-cancer-detection/train'

In [5]:
labels['id']=labels['id'].apply(lambda x: x+'.tif')
labels.head()

NameError: name 'labels' is not defined

In [6]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(preprocessing_function=lambda x:(x - x.mean()) / x.std() if x.std() > 0 else x,
                                   horizontal_flip=True,
                                   vertical_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=lambda x:(x - x.mean()) / x.std() if x.std() > 0 else x)


ModuleNotFoundError: No module named 'keras'

In [7]:
datagen = ImageDataGenerator(
       horizontal_flip=True,
       vertical_flip=True,
       brightness_range=[0.5, 1.5],
       fill_mode='reflect',                               
       #rotation_range=15,
       rescale=1./255,
       #shear_range=0.2,
       #zoom_range=0.2,
       validation_split=0.2)

NameError: name 'ImageDataGenerator' is not defined

In [8]:
tragen= datagen.flow_from_dataframe(
    dataframe= labels,
    directory= train,
    x_col= 'id',
    y_col= 'label',
    #has_ext= False,
    batch_size= 64,
    shuffle= True,
    #validate_filenames=True,
    subset= 'training',
    seed= 2020,
    class_mode= 'binary',
    target_size= (96,96)
)

validgen=datagen.flow_from_dataframe(
    dataframe= labels,
    directory= valid,
    x_col= 'id',
    y_col= 'label',
    #has_ext= False,
    batch_size= 32,
    shuffle= False,
    #validate_filenames=True,
    subset= 'validation',
    seed= 2020,
    class_mode= 'binary',
    target_size= (96,96))

testgen=datagen.flow_from_dataframe(
    dataframe= labels,
    directory= train,
    x_col= 'id',
    y_col= 'label',
    #has_ext= False,
    batch_size= 32,
    shuffle= False,
    #validate_filenames=True,
    subset= 'validation',
    seed= 2020,
    class_mode= 'binary',
    target_size= (96,96))



NameError: name 'datagen' is not defined

In [9]:
tragen.class_indices['0']= 'Normal'
tragen.class_indices['1']= 'Cancer'
tragen.class_indices

NameError: name 'tragen' is not defined

In [10]:
validgen.class_indices['0']= 'Normal'
validgen.class_indices['1']= 'Cancer'
validgen.class_indices

NameError: name 'validgen' is not defined

In [11]:
testgen.class_indices['0']= 'Normal'
testgen.class_indices['1']= 'Cancer'
testgen.class_indices

NameError: name 'testgen' is not defined

In [12]:
model= Sequential()
model.add(Conv2D(16,(3,3), activation= 'relu', input_shape=(96,96,3)))
model.add(BatchNormalization())
model.add(Conv2D(16,(3,3), activation= 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size= (2,2)))
model.add(Dropout(0.2))


model.add(Conv2D(32,(3,3), activation= 'relu',use_bias= False))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3), activation= 'relu', use_bias= False))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size= (2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3), activation= 'relu',use_bias= False))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3), activation= 'relu', use_bias= False))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size= (2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(96,(3,3), activation= 'relu', use_bias= False))
model.add(BatchNormalization())
model.add(Conv2D(96,(3,3), activation= 'relu', use_bias= False))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size= (2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation= "sigmoid"))




NameError: name 'Sequential' is not defined

In [13]:
model.summary()

NameError: name 'model' is not defined

In [14]:
model.compile(optimizer= Adam(0.001),
              loss= 'binary_crossentropy',
              metrics= ["accuracy"])

NameError: name 'model' is not defined

In [15]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

STEP_SIZE_TRAIN=tragen.n//tragen.batch_size
STEP_SIZE_VALID=validgen.n//validgen.batch_size
STEP_SIZE_TEST=testgen.n//testgen.batch_size

ModuleNotFoundError: No module named 'keras'

In [16]:
earlystop = EarlyStopping(monitor='val_loss', patience=2, verbose=1,
                             mode= 'max', restore_best_weights=True)
reduce = ReduceLROnPlateau(monitor='val_loss', patience=1,
                            verbose=1,mode= 'max', factor=0.1, min_lr= 0.001)

history= model.fit_generator(tragen, steps_per_epoch= STEP_SIZE_TRAIN,
                             validation_data= validgen,
                             validation_steps= STEP_SIZE_VALID, epochs= 20,
                             callbacks= [earlystop, reduce])

NameError: name 'EarlyStopping' is not defined

In [17]:
model.metrics_names

NameError: name 'model' is not defined

In [18]:
tragen.class_indices

NameError: name 'tragen' is not defined

In [19]:
val_loss, val_acc = \
model.evaluate_generator(testgen, 
                        steps=STEP_SIZE_TEST)

print('val_loss:', val_loss)
print('val_acc:', val_acc)

NameError: name 'model' is not defined

In [20]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

NameError: name 'history' is not defined

In [ ]:
import time
t= time.time()
export= './{}.h5'.format(int(t))
print(export)

model.save(export)